## <font color = "brown"> Importing Libraries </font>

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings as wn

wn.filterwarnings("ignore")

In [2]:
df = pd.read_excel("../../raw_data/market_data.xlsx") # Open original data
df = df[["Time (UTC+10)", "Regions VIC Trading Price ($/MWh)"]] # Select time and victoria prices
df.columns = ["Time", "Price"] # Rename columns
df["Time"] = pd.to_datetime(df["Time"]) # Convert data type
df = df.sort_values("Time").reset_index(drop = True) # Finalise

## <font color = "brown"> Moving Average Model </font>

The moving average (MA) model calculates the centered moving average line of the time series. This model takes one parameter $n$, which is the number of data points back and forth to be considered in the centered moving average.

The centered moving average in mathematical terms is,

$$
\begin{aligned}
\text{CMA}_n(y_t) &= \frac{y_t + \sum\limits_{i=1}^{n}\left[y_{t-i} + y_{t+1}\right]}{2n + 1} \\
&= \frac{y_t + (y_{t-1} + y_{t-2} + \dots + y_{t-n}) + (y_{t+1} + y_{t+2} + \dots + y_{t+n})}{2n + 1}
\end{aligned}
$$

Where $y_t$ represents the spot price at any time $t$. Hence technically, this average is a $(2n + 1)$ simple moving average for $y_{t+n}$, or the $(2n+1)$ centered moving average for $y_t$. Since we are using it for time $t$, we will call it a centered moving average.


In [3]:
def CMA(df, n):
    
    # Defining variables
    n = n + 1
    series = df["Price"]
    
    # Calculate (n+1)-simple moving average looking back
    BackSum = series.rolling(n).sum()
    series = series.iloc[::-1]
    
    # Calculate (n+1)-simple moving average looking ahead
    FrontSum = series.rolling(n).sum()
    series = series.iloc[::-1]
    
    # Calculate the (2n+1)-centered moving average
    df["MA"] = (BackSum + FrontSum - series) / (2 * n - 1)
    
    return df.dropna().reset_index(drop = True)

In [4]:
def VisualiseMA(df, ma):
    
    # Visualise the calculated moving averages
    sns.lineplot(y = "Price", x = "Time", data = df)
    sns.lineplot(y = "MA", x = "Time", data = ma)

In [5]:
def Divide(df):
    
    # Defining data points above the moving average
    UpperPrice = df[df.Price >= df.MA]
    UpperPrice["Status"] = "Discharge"
    
    # Defining data points below the moving average
    LowerPrice = df[df.Price < df.MA]
    LowerPrice["Status"] = "Charge"
    
    return LowerPrice, UpperPrice

In [6]:
def BiggerPicture(df, Charge, Discharge):
    
    # Function to assign action to each time point
    cdc = pd.concat([Charge, Discharge], ignore_index = True)

    # Join with original dataframe
    dff = pd.merge(df, cdc[["Time", "Status"]], on = "Time", how = "left")
    
    # Select and rename columns
    dff = dff[["Time", "Price", "Status"]]
    dff.columns = ["Time", "Price", "Status"]
    
    return dff

In [7]:
def MovingAverage(n, df = df):
    
    # Create Moving Average Price Data
    PriceMA = CMA(df.copy(), n)
    
    #Visualise Moving Averages (Do not visualise for entire dataset)
    #VisualiseMA(df, PriceMA)
    
    # Divide Price Data
    LowerPrice, UpperPrice = Divide(PriceMA)
    
    # Assign Action to Time Point
    final = BiggerPicture(df.copy(), LowerPrice, UpperPrice)
    
    return final

In [28]:
M = MovingAverage(17)

## <font color = "brown"> Region Maximisation Model </font>

The region maximisation (RM) model takes a region of consecutive actions and the opening capacity, and returns the most effective way to charge or discharge. We define a 'region of consecutive actions' as time periods where the previous models recommend consecutive actions.

<table>
<tr>
    <th> Chronological dispatches </th><th> RM Dispatches</th>
</tr>
<tr>
    <td>
        <table>
            <tr><th> Time </th><th> Action </th><th>  ROCA </th><th>  Price </th><th> Dispatch</tr>
            <tr><td> 2120-01-01 00:00:00 </td><td> Charge </td><td> 1 </td><td> \$20 </td><td> -150 </td></tr>
            <tr><td> 2120-01-01 00:30:00 </td><td> Charge </td><td> 1 </td><td> \$24 </td><td> -150 </td></tr>
            <tr><td> 2120-01-01 01:00:00 </td><td> Charge </td><td> 1 </td><td> \$23 </td><td> -150 </td></tr>
            <tr><td> 2120-01-01 01:30:00 </td><td> Charge </td><td> 1 </td><td> \$21 </td><td> -150 </td></tr>
            <tr><td>2120-01-01 02:00:00 </td><td> Charge </td><td> 1 </td><td> \$18 </td><td> -44 </td></tr>
            <tr><td>2120-01-01 02:30:00 </td><td> Charge </td><td> 1 </td><td> \$17 </td><td> 0 </td></tr>
            <tr><td>2120-01-01 03:00:00 </td><td> Discharge </td><td> 2 </td><td> \$31 </td><td> 135 </td></tr>
            <tr><td>2120-01-01 03:30:00 </td><td> Discharge </td><td> 2 </td><td> \$30 </td><td> 135 </td></tr>
            <tr><td>2120-01-01 04:00:00 </td><td> Discharge </td><td> 2 </td><td> \$29 </td><td> 135 </td></tr>
            <tr><td>2120-01-01 04:30:00 </td><td> Discharge </td><td> 2 </td><td> \$32 </td><td> 135 </td></tr>
            <tr><td>2120-01-01 05:00:00 </td><td> Discharge </td><td> 2 </td><td> \$33 </td><td> 117 </td></tr>
            <tr><td>2120-01-01 05:30:00 </td><td> Discharge </td><td> 2 </td><td> \$34 </td><td> 0 </td></tr>
        </table>
    </td>
    <td>
        <table>
            <tr><th> Time </th><th> Action </th><th>  ROCA </th><th>  Price </th><th> Dispatch</tr>
            <tr><td> 2120-01-01 00:00:00 </td><td> Charge </td><td> 1 </td><td> \$20 </td><td> -150 </td></tr>
            <tr><td> 2120-01-01 00:30:00 </td><td> Charge </td><td> 1 </td><td> \$24 </td><td> 0 </td></tr>
            <tr><td> 2120-01-01 01:00:00 </td><td> Charge </td><td> 1 </td><td> \$23 </td><td> -44 </td></tr>
            <tr><td> 2120-01-01 01:30:00 </td><td> Charge </td><td> 1 </td><td> \$21 </td><td> -150 </td></tr>
            <tr><td>2120-01-01 02:00:00 </td><td> Charge </td><td> 1 </td><td> \$18 </td><td> -150 </td></tr>
            <tr><td>2120-01-01 02:30:00 </td><td> Charge </td><td> 1 </td><td> \$17 </td><td> -150 </td></tr>
            <tr><td>2120-01-01 03:00:00 </td><td> Discharge </td><td> 2 </td><td> \$31 </td><td> 135 </td></tr>
            <tr><td>2120-01-01 03:30:00 </td><td> Discharge </td><td> 2 </td><td> \$30 </td><td> 117 </td></tr>
            <tr><td>2120-01-01 04:00:00 </td><td> Discharge </td><td> 2 </td><td> \$29 </td><td> 0 </td></tr>
            <tr><td>2120-01-01 04:30:00 </td><td> Discharge </td><td> 2 </td><td> \$32 </td><td> 135 </td></tr>
            <tr><td>2120-01-01 05:00:00 </td><td> Discharge </td><td> 2 </td><td> \$33 </td><td> 135 </td></tr>
            <tr><td>2120-01-01 05:30:00 </td><td> Discharge </td><td> 2 </td><td> \$34 </td><td> 135 </td></tr>
        </table>
    </td>
</tr>


The left table shows market dispatches formulated using chronological order, while the right table shows dispatch formulated by the RM model. We can see in any of the fictitious table above, how consecutive time points with the same actions are categorised as regions of consecutive actions (ROCA). Changing actions would change the ROCA category, hence the name 'region of **consecutive** actions'. 

In the process of filling up market dispatches, we notice that using chronological order to fill dispatches, most of the time, yields inefficient results. As we can see for both ROCAs in the left table, the 2 best price points in each ROCAs are given the least amount of dispatches. This would decrease the total revenue in the final calculation. The RM model solves this by maximising each ROCAs based on their current capacities, action (charge/discharge) and most importantly, the prices. In other words, it allocates the best price points to the most dispatches. The table on the right shows how the dispatches move, when the RM model is applied.

In [8]:
def FindActions(df):
    
    # Separate periods of action and no action
    action = df[df["Status"] != "Do Nothing"]

    # Group action periods based on consecutive action
    action["Period"] = action["Status"].ne(action["Status"].shift()).cumsum()
    action["Vec"] = np.where(action["Status"] == "Discharge", action["Price"], -action["Price"])
    
    return action

In [9]:
def Limit(capacity, status, end = 0):
    
    # Limit possible dispatch by looking at capacity and action
    if status == "Charge":
        if capacity + 135 > 580:
            # Maximum charge
            restrict = 580 - capacity
        else:
            restrict = 135
    else:
        if capacity - 150 < end:
            # Maximum discharge
            restrict = end - capacity
        else:
            restrict = -150
            
    return restrict

def Chronological(action, start = 0, end = 0):
    # Allocate dispatch based on chronological order
    
    # Set initial energy conditions
    action["Opening Capacity"] = 0
    action["Closing Capacity"] = 0

    # Declare additional energy variables
    action = action.sort_values("Time").reset_index(drop = True)
    action["Opening Capacity"].loc[0] = start
    action["Restrict"] = np.nan
    
    # Looping to set opening and closing capacities
    for i in action.index:
        data = action.loc[i]
        
        # Limiting possible energy dispatch
        action["Restrict"].loc[i] = Limit(data["Opening Capacity"], data["Status"], end)
        
        # Closing capacity = Opening capacity + activities
        action["Closing Capacity"].loc[i] = action["Opening Capacity"].loc[i] + action["Restrict"].loc[i]
        try:
            # Set next entry's opening capacity to current entry's closing capacity
            action["Opening Capacity"].loc[i + 1] = action["Closing Capacity"].loc[i]
        except:
            # Exception if last data
            pass
        
    action["Actual"] = np.where(action["Status"] == "Discharge", action["Restrict"] * 0.9, action["Restrict"] / 0.9)
    
    return action

In [10]:
def Maximise(action):
    
    # Divide dataframe based on consecutive actions
    periods = [v for k, v in action.groupby(["Period"])]
    
    # Looping through each consecutive action
    for i in range(len(periods)):
        data = periods[i]
        
        # Only process if actions are not limited to 0 due to capacity
        if data["Restrict"].sum() != 0:
            
            # Assign preferable SPs higher market dispatch
            restrict = sorted(abs(data["Restrict"]).tolist(), reverse = True)
            data = data.sort_values("Vec", ascending = False)
            data["Restrict"] = restrict
            periods[i] = data
            
    # Rejoin all periodic dataframe
    try:
        action = pd.concat(periods)
    except:
        action = pd.DataFrame(columns = action.columns)
    action = action.sort_values("Time").reset_index(drop = True)
    
    # Reassign market dispatch number
    action["Actual"] = -np.where(action["Status"] == "Discharge", action["Restrict"] * 0.9, action["Restrict"] / 0.9)
    
    return action

In [11]:
def PostProcess(action, df):
    
    # Select columns required for revenue calculation
    action = action[["Time", "Price", "Status", "Actual", "Restrict", "Opening Capacity", "Closing Capacity"]]
    
    # Reverse the price because we used the opposite sign to maximise
    action["Restrict"] = np.where(action["Status"] == "Charge", abs(action["Restrict"]), -abs(action["Restrict"]))
    action["Actual"] = np.where(action["Status"] == "Charge", -abs(action["Actual"]), abs(action["Actual"]))
    
    # Remove no-dispatch actions
    action = action[action["Restrict"] != 0]
    df = pd.merge(df, action, on = ["Time", "Price"], how = "left")
    
    # Imputation
    df["Status"] = df["Status"].replace(np.nan, "Do Nothing")
    df["Actual"] = df["Actual"].replace(np.nan, 0)
    df["Restrict"] = df["Restrict"].replace(np.nan, 0)
    
    return df

In [12]:
def Maximisation(MX, df = df, start = 0, end = 0, Chronos = True):
    
    # Separate action and no action
    action = FindActions(MX.dropna())
    
    # Fill out dispatch values based on the energy restrictions
    if Chronos:
        action = Chronological(action, start, end)
    
    # Maximise allocated dispatch value
    action = Maximise(action)
    
    # Post processing
    action = PostProcess(action, df)
    
    return action.reset_index(drop = True)

In [34]:
MX = Maximisation(M)

## <font color = "brown"> Loss Removal Model </font>

The loss removal (LR) model looks at transactions between 2 ROCAs and see if there are any matching transactions, where it is actually not profitable to conduct those transactions. This model makes use of the fact that ROCAs are always switching in action by definition. Hence we can compare consecutive ROCAs and find unprofitable transactions.

Due to the MA and RM model, it is almost guaranteed that discharging ROCA's price points are always higher than their preceeding charging ROCA's price points. But due to the efficiency and the Marginal loss factor (MLF) in the revenue calculations, higher alone is not enough.

When charging takes place in a time point $t$, at $C$ Mwh, the actual increase in capacity is only $D = C \times 0.9$ accounting for efficiency. When dispatching this in the next time point, the maximum possible dispatched amount is $D \times 0.9$ accounting to efficiency. At the end, the revenue from charge $C$ is only obtained from $C \times 0.9^2$. We want to know whether the charging-discharging pairs of points from consecutive ROCAs passes the breakeven point or not. If not, we will discard these points.

Let $P_c$ be the charging price, $P_d$ be the discharging price, and $R$ be the revenue. The breakeven point is a price combination point, where the revenue will be 0.

$$
\begin{aligned}
R &= D P_d \times 0.991 - C P_c \times \frac{1}{0.991} \\
0 &=  0.9^2 C P_d \times 0.991 - C P_c \times \frac{1}{0.991} \\
0.80271 P_d &= \frac{P_c}{0.991} \\
P_d &= 1.257P_c
\end{aligned}
$$

Hence to translate this, we want to filter out every charging-discharging pair where the discharging price $P_d$ is less than $1.257P_c$ and keep if otherwise.

The process of filtering out actions is a process of elimination. Every charging ROCA will have a succeeding discharging ROCA. We will compare these succeding pairs of charges and discharges. We define restricted dispatch $RD$ as the market dispatch, $MD$, multiplied by the efficiency factor.

$$
\begin{aligned}
RD &=
\begin{cases}
       MD\div0.9 & \text{for Action = Charge}  \\
       MD\times0.9 & \text{for Action = Discharge}  \\
\end{cases}
\end{aligned}
$$

The algorithm of this model takes the least profitable pairs of charge $C_t$ and discharge $D_t$, and discard them if not profitable. The process accounts for their $RD$, which means if the value of $RD$ does not match between the compared pairs, we will remove  the action with least $RD$ and subtract the other action by $\min(C_t, D_t)$. The action that remains (first anchor point) will then be compared to the next worst competing action. The process stops if at any comparison, the discharge price exceeds the breakeven point.

In [13]:
def LRProcess(df):
    
    # Preprocessing
    df = df.drop(["Opening Capacity", "Closing Capacity"], axis = 1)
    df = df[df["Status"] != "Do Nothing"]
    df["Period"] = df["Status"].ne(df["Status"].shift()).cumsum()
    
    return df.reset_index(drop = True)

In [14]:
def Breakeven(price):
    
    # Returns the discharge price to breakeven
    return price/(0.9 * 0.9 * 0.991 * 0.991)

def RemoveLoss(sdf):
    
    # Sort charging ROCA based on worst (highest) price
    charge = sdf[sdf["Status"] == "Charge"].sort_values("Price", ascending = False).reset_index(drop = True)
    
    # Sort discharging ROCA based on worst (lowest) price
    discharge = sdf[sdf["Status"] == "Discharge"].sort_values("Price", ascending = True).reset_index(drop = True)

    try:
        # Initialise worst charge and discharge price
        CP = charge["Price"].loc[0]
        DP = discharge["Price"].loc[0]
        
        # Initialise charge and discharge dispatch
        disrestrict = discharge["Restrict"].loc[0]
        charestrict = charge["Restrict"].loc[0]
        
    except KeyError:
        # Exception if last charging ROCA does not have
        # a Succeeding competing ROCA
        return charge
    
    # The order of the first absolute sum determines 
    # which ROCA gets treated as the initial anchor point
    if abs(disrestrict) > abs(charestrict):
        
        # Treats first discharge as the first anchor point
        first = discharge
        second = charge
        pos = "Dis" # For convenience further in
        
    else:
        # Treats first charge as the first anchor point
        first = charge
        second = discharge
        pos = "Char" # For convenience further in
       
    # Iteration Variables
    i = 0
    position = "First"
        
    # Loop stays until breakeven is achieved at any point.
    # By definition, if the n-th worst charge and discharge pairs
    # have breakeven, all remaining pair will also exceed breakeven.
    while DP < Breakeven(CP):
        
        # The variable 'position' tells the code on which comparison to make
        # First: compares the i-th action against i-th competing action
        if position == "First":
            
            # Define compared dispatches
            fres = first["Restrict"].loc[i]
            sres = second["Restrict"].loc[i]

            # Drop the action with least dispatch
            second = second.drop(i)
            
            # Update the action with greater dispatch
            first["Restrict"].loc[i] = fres + sres
            
            try:
                # Set next worst action as next comparison
                if pos == "Dis":
                    DP = first["Price"].loc[i + 1]
                else:
                    DP = second["Price"].loc[i + 1]
            except:
                # Exception if last entry
                break
            
            # Update position
            position = "Second"
            
            # Iteration only updates when position = 'First'
            i += 1
            
        # Second: Compares the i-th action and (i-1)-th competing action
        elif position == "Second":
            
            # Define compared dispatches
            fres = first["Restrict"].loc[i - 1]
            sres = second["Restrict"].loc[i]
            
            # Drop the action with least dispatch
            first = first.drop(i - 1)
            
            # Update the action with greater dispatch
            second["Restrict"].loc[i] = fres + sres
            
            try:
                # Set next worst action as next comparison
                if pos == "Dis":
                    CP = second["Price"].loc[i + 1]
                else:
                    CP = first["Price"].loc[i + 1]
            except:
                # Exception if last entry
                break
            
            # Update position
            position = "First"
            
        # Stop if any ROCA runs out of actions
        if min(len(first), len(second)) == 0:
            break
    
    # Retain original charging dataframe
    if pos == "Dis":
        charge = second
    else:
        charge = first

    if pos == "Dis":
        second = charge
        #first = first[first["Restrict"] != 0]
        #second = second[second["Restrict"] >= 0]
    else:
        first = charge 
        #first = first[first["Restrict"] >= 0]
        #second = second[second["Restrict"] != 0]
        
    # Concat all processed dataframes
    sdf = pd.concat([first, second])
    
    return sdf

def CompareROCA(df):
    
    # Find all unique ROCAs
    ROCAs = df["Period"].unique().tolist()
    dflist = []
    
    # Loop through all charging ROCAs
    for i in [x for x in ROCAs if x % 2 == 1]:
        subdf = df[(df["Period"] == i) | (df["Period"] == i + 1)]
        subdf = RemoveLoss(subdf) # Main algorithm function
        dflist.append(subdf)
        
    # Concating all processed sub-dataframes
    findf = pd.concat(dflist, ignore_index = True).sort_values("Time")
    
    return findf

In [15]:
def LossRemoval(df):
    
    # Preprocessing
    rldf = LRProcess(df.copy())
    
    # Remove Losses
    rldf = CompareROCA(rldf)
    
    return FillCapacity(rldf)

In [16]:
def FillCapacity(df):

    cdf = df.copy().reset_index(drop = True)
    
    # Filling closing and opening capacities with dispatch given
    cdf["Opening Capacity"] = 0
    cdf["Closing Capacity"] = 0

    for i in cdf.index:
        # Closing capacity = Opening capacity + activities
        if cdf["Status"].loc[i] == "Charge":
            cdf["Closing Capacity"].loc[i] = cdf["Opening Capacity"].loc[i] + abs(cdf["Restrict"].loc[i])
        else:
            cdf["Closing Capacity"].loc[i] = cdf["Opening Capacity"].loc[i] - abs(cdf["Restrict"].loc[i])
            
        try:
            # Set next entry's opening capacity to current entry's closing capacity
            cdf["Opening Capacity"].loc[i + 1] = cdf["Closing Capacity"].loc[i]
        except:
            # Exception if last data
            pass
        
    # Recopy value to actual dataframe
    df["Opening Capacity"] = cdf["Opening Capacity"].tolist()
    df["Closing Capacity"] = cdf["Closing Capacity"].tolist()
    
    # Edit actual dispatch based on edited actions
    df["Actual"] = -np.where(df["Status"] == "Discharge", df["Restrict"] * 0.9, df["Restrict"] / 0.9)
    
    return df

def Prepare(df, N):
    
    # Simple preprocessing
    N = N[N["Actual"] != 0]
    
    ND = pd.merge(df, N[["Time", "Price", "Status", "Actual", "Restrict", "Opening Capacity", "Closing Capacity"]], 
                  on = ["Time", "Price"], how = "left")
    
    # Imputation
    ND["Status"] = ND["Status"].replace(np.nan, "Do Nothing")
    ND["Actual"] = ND["Actual"].replace(np.nan, 0)
    ND["Restrict"] = ND["Restrict"].replace(np.nan, 0)
    
    # Fix Capacities
    return FillCapacity(ND)

In [39]:
LR = LossRemoval(MX)

## <font color = "brown"> Stationary Maximisation Model</font>

The stationary maximisation (SM) model looks at all the time points where previous models ignores (which we will call stationary points) and ask the question, "Could there be revenue in these time points?". To understand how this model works,  we need to know how the revenue calculation works.


When charging takes place in a time point $t$, at $C$ Mwh, the actual increase in capacity is only $D = C \times 0.9$ accounting for efficiency. When dispatching this in the next time point, the maximum possible dispatched amount is $D \times 0.9$ accounting to efficiency. At the end, the revenue from charge $C$ is only obtained from $C \times 0.9^2$.

Making use of this efficiency fact, let $P_c$ be the theoretical charging price, $P_d$ be the theoritical discharging price and $R$ be the revenue on any time point. If we want to know whether revenue can be obtained from stationary points, we have to calculate the theoritical revenue, given that we know both charge and discharge prices.

$$
\begin{aligned}
R &= D P_d \times 0.991 - C P_c \times \frac{1}{0.991} \\
&=  0.9^2 C P_d \times 0.991 - C P_c \times \frac{1}{0.991} \\
&= (0.80271 P_d -  \frac{P_c}{0.991})C
\end{aligned}
$$

Notice that the calculation for revenue is now a scalar multiple of the initial charging dispatch. This means that we can just set $C = 1$ and see if the revenue is positive or not to know whether any other value of $C > 0$ is also profitable. Hence the SM model calculates the theoretical revenue,

$$
\begin{aligned}
TR &= 0.80271 P_d -  \frac{P_c}{0.991}
\end{aligned}
$$

Where $P_c = P_t$ and $P_d = P_{t+1}$, in which $t$ indicates the current time iteration. The model loops for every stationary point $t$, and assign charging and discharging status respectively at time $t$ and $t+1$ if the $TR$ at time $t$ is $>0$. Because the revenue is a scalar multiple of $C$, we can maximise the revenue by maximising $C$, hence the amount of dispatch will be the maximum possible given the current opening capacity at time $t$.

In [17]:
def Profit(D, C, charge):
    
    # Function to find theoritical profit given the 
    # charge dispatch, charge price, and discharge price
    discharge = charge * 0.9 * 0.9
    profit = D * discharge * 0.991 - C * charge / 0.991
    
    return profit

def SMProcess(df):
    
    # Function to create features based on theoritical revenue calculations
    D = df.loc[df["Status"] == "Do Nothing"].sort_values("Time")
    D["Datediff"] = (D["Time"] - D["Time"].shift()).dt.total_seconds()/60
    D["NextPrice"] = D["Price"].shift(-1)
    D["Theoritical"] = Profit(D["NextPrice"], D["Price"], 150)
    
    return D

In [18]:
def Reassign(D):
    
    # Charge on theoritically profitable time points
    D["Status"] = np.where(D["Theoritical"] > 0, "Charge", D["Status"])
    D["Previous Status"] = D["Status"].shift()
    D["Previous Index"] = D.index
    D["Previous Index"] = D["Previous Index"].shift()

    for i in D.index:
        data = D.loc[i]
                
        # Cancel action on non-consecutive time periods
        if data["Previous Status"] == "Charge" and data["Datediff"] != 30:
            D["Status"].loc[data["Previous Index"]] = "Do Nothing"
            
        # Cancel action if capacity is full
        elif data["Previous Status"] == "Charge" and D["Opening Capacity"].loc[data["Previous Index"]] == 580:
            D["Status"].loc[data["Previous Index"]] = "Do Nothing"
            
        # Reassign action if all conditions fulfilled
        elif data["Previous Status"] == "Charge" and data["Datediff"] == 30:
            
            # Set balancing discharge action
            D["Status"].loc[i] = "Discharge"
            
            # Update charging information
            D["Restrict"].loc[data["Previous Index"]] = abs(Limit(D["Opening Capacity"].loc[data["Previous Index"]], "Charge"))
            D["Actual"].loc[data["Previous Index"]] = -D["Restrict"].loc[i] / 0.9
            D["Closing Capacity"].loc[data["Previous Index"]] = D["Opening Capacity"].loc[data["Previous Index"]] + D["Restrict"].loc[data["Previous Index"]]
            
            # Update discharging information
            D["Opening Capacity"].loc[i] = D["Closing Capacity"].loc[data["Previous Index"]]
            D["Restrict"].loc[i] = - D["Restrict"].loc[data["Previous Index"]]
            D["Actual"].loc[i] = -D["Restrict"].loc[i] * 0.9
            D["Closing Capacity"].loc[i] = D["Opening Capacity"].loc[i] + D["Restrict"].loc[i]
            
            # Update action if consecutive charging appears
            if data["Status"] == "Charge":
                D["Previous Status"] = D["Status"].shift()
    
    # Filter actions only
    D = D[(D["Status"] != "Do Nothing")]
    
    return D, D.index
    

In [19]:
def Stationary(N, df = df):
    
    # Preprocessing
    Model = Prepare(df, N)
    
    # Feature Engineering
    Station = SMProcess(Model)
    # Algorithm to find theoritically profitable transactions
    Station, index = Reassign(Station)
    
    # Concating the results from this model with previous model
    Fin = pd.concat([Model.drop(index), Station[Model.columns]]).sort_values("Time")
    Fin["Actual"] = - np.where(Fin["Status"] == "Discharge", Fin["Restrict"] * 0.9, Fin["Restrict"] / 0.9)
    
    return Fin

In [43]:
SM = Stationary(LR)

## <font color = "brown"> Action Shift Model </font>

The action shift (AS) model is an extension of the RM model. The RM model allocates most dipatches to the best prices in a ROCA. The AS model shifts charging or discharging actions to the best price in a region of consecutive non-actions and actions (ROCNA). 

A ROCNA is classified into a charging ROCNA and a discharging ROCNA. The formal definition of a ROCNA is a region between 2 opposing actions. i.e Let's say that we want to look for a charging ROCNA at any point in the data. If a discharge action is present at time $t$ and the next discharging action is located at time $t + n$, then the period $[t + 1, t + n -1]$ is classified as a charging ROCNA period. This is also applicable for a discharging ROCNA, where we are now looking for periods between 2 charging actions non-inclusive. Note that a time point is not exclusive to one ROCNA, as it can be part of 2 ROCNAs maximum. The table below shows the movement of some ROCNAs.

<table>
    <tr><th> Time </th><th> Action </th><th>  ROCNACharge </th><th>  ROCNADischarge </th></tr>
    <tr><td> 2120-01-01 00:00:00 </td><td> Charge </td><td> 1 </td><td>  </td></tr>
    <tr><td> 2120-01-01 00:30:00 </td><td> Do Nothing </td><td> 1 </td><td>  </td></tr>
    <tr><td> 2120-01-01 01:00:00 </td><td> Charge </td><td> 1 </td><td>  </td></tr>
    <tr><td> 2120-01-01 01:30:00 </td><td> Do Nothing </td><td> 1 </td><td> 1 </td></tr>
    <tr><td>2120-01-01 02:00:00 </td><td> Discharge </td><td>  </td><td> 1 </td></tr>
    <tr><td>2120-01-01 02:30:00 </td><td> Discharge </td><td>  </td><td> 1 </td></tr>
    <tr><td>2120-01-01 03:00:00 </td><td> Do Nothing </td><td> 2 </td><td> 1 </td></tr>
    <tr><td>2120-01-01 03:30:00 </td><td> Do Nothing </td><td> 2 </td><td> 1 </td></tr>
    <tr><td>2120-01-01 04:00:00 </td><td> Charge </td><td> 2 </td><td>  </td></tr>
    <tr><td>2120-01-01 04:30:00 </td><td> Do Nothing </td><td> 2 </td><td> 2 </td></tr>
    <tr><td>2120-01-01 05:00:00 </td><td> Do Nothing </td><td> 2 </td><td> 2 </td></tr>
    <tr><td>2120-01-01 05:30:00 </td><td> Do Nothing </td><td> 2 </td><td> 2 </td></tr>
    <tr><td>2120-01-01 06:00:00 </td><td> Discharge </td><td>  </td><td> 2 </td></tr>
    <tr><td>2120-01-01 06:30:00 </td><td> Do Nothing </td><td>  </td><td> 2 </td></tr>
</table>

The SA model looks at each ROCNAs chronologically by their starting time and shift around the actions to their maximised version. In order to preserve capacity level, a restriction must be put in place after every ROCNA shifts. If a shift takes place in the previous opposing ROCNA, then the time points considered for the current iteration's maximisation can only take place in the time points above the maximum (time) action in the previous ROCNA shifts.

In mathematical terms, if a single ROCNA $R_k$ has actions on time $\{a, b, c, d, e\}$ and goes through the SA model,

$$
\begin{aligned}
\{y_a, y_b, y_c, y_d, y_e\} &\xrightarrow[\text{}]{\text{Shift}} \{y_{a'}, y_{b'}, y_{c'}, y_{d'}, y_{e'}\}
\end{aligned}
$$

To analyse the next ROCNA $R_{k+1}$, we want to look at the maximum time of the shifted actions,

$$
\begin{aligned}
T &= \text{Max}_{Time}(y_{a'}, y_{b'}, y_{c'}, y_{d'}, y_{e'})
\end{aligned}
$$

Then, analysing $R_{k+1}$ can not be done naïvely on the entire ROCNA due to capacity limitations. To maintain capacity restrictions, the next shifting region takes place only on the time points $t$ that satisfies,

$$
\begin{aligned}
(t \in R_{k+1}) \cap (t > T)
\end{aligned}
$$

This process is iterated for all charging and discharging ROCNAs.

In [20]:
def ROCNA(df, status, other):

    # Finding ROCNA periods
    W = df[df["Status"] == status]["Period"].unique()
    W = np.sort(np.unique(np.array(list(W) + list(W+1) + list(W-1))))
    
    # Finding ROCNA limitations
    Banned = df[df["Status"] == other]["Period"].unique()
    W = W[~np.isin(W, Banned)]

    # Merge ROCNA column to initial dataframe
    Com = pd.DataFrame()
    Com["Period"] = W
    Com[f"ROCNA{status}"] = ((Com["Period"] - Com["Period"].shift()) != 1).cumsum()
    fin = pd.merge(df, Com, on = "Period", how = "left")

    return fin

def ASProcess(df):
    
    # Assign ROCA
    df = df[["Time", "Price", "Status", "Actual", "Restrict", "Opening Capacity", "Closing Capacity"]]
    df["Period"] = df["Status"].ne(df["Status"].shift()).cumsum()
    
    # Assign ROCNA from ROCAs
    Actions = ["Charge", "Discharge"]
    for i in range(len(Actions)):
        df = ROCNA(df, Actions[i], Actions[i - 1])
    
    # Create column for maximum time restrictions
    df["Shift"] = False
    
    return df

In [21]:
def Shift(df):
    
    # Main algorithm function
    
    # Iteration variable for each 
    # charging and discharging ROCNA
    i, j = 1, 1
    
    # First ROCA is always charging
    current = "Charge"
    
    # Loop until reaching last charge and discharge ROCNAs
    while (i < df["ROCNACharge"].max() and j < df["ROCNADischarge"].max()):
        
        # Current ROCNA action
        stat = current
        
        # Action for charging ROCNA
        if current == "Charge":
            
            # ROCNACharge: Filtering iteration's charging ROCNA
            # Shift: Filter time points after maximum time shift
            subdf = df[(df["ROCNACharge"] == i) & (~df["Shift"])]
            
            # Sort by best prices to worst (lowest to highest)
            subdf = subdf.sort_values("Price", ascending = True)
            
            # Increase charge iteration
            i += 1
            
            # Change next action
            current = "Discharge"
            
        # Action for discharging ROCNA
        elif current == "Discharge":
            # ROCNACharge: Filtering iteration's discharging ROCNA
            # Shift: Filter time points after maximum time shift
            subdf = df[(df["ROCNADischarge"] == j) & (~df["Shift"])]
            
            # Sort by best prices to worst (highest to lowest)
            subdf = subdf.sort_values("Price", ascending = False)
            
            # Increase charge iteration
            j += 1
            
            # Change next action
            current = "Charge"
            
        # Skip next code if length of ROCNA is 1
        if len(subdf) == 1:
            continue
            
        # Shift best dispatches to best prices
        Dispatches = sorted([abs(x) for x in subdf["Restrict"].tolist()], reverse = True)
        subdf["Restrict"] = Dispatches
        subdf["Status"] = np.where(subdf["Restrict"] > 0, stat, "Do Nothing")
        
        # Record maximum time after shift
        subdf["Shift"] = np.where((subdf["Time"] <= subdf[subdf["Restrict"] > 0]["Time"].max())
                                  , True, False)
        
        # Replace ROCNA with shifted ROCNA
        df.iloc[subdf.index] = subdf
       
    return df

In [22]:
def ASPost(df):
    
    # Fix dispatch and efficiency calculation and sign
    df["Restrict"] = np.where(df["Status"] == "Discharge", -abs(df["Restrict"]), abs(df["Restrict"]))
    df["Actual"] = - np.where(df["Status"] == "Discharge", df["Restrict"] * 0.9, df["Restrict"] / 0.9)
    
    return df.sort_values("Time")

In [23]:
def ShiftAction(df):
    
    # Assign ROCNAs
    SA = ASProcess(df.copy())
    
    # Shift actions in ROCNAs
    SA = Shift(SA)
    
    # Post processing
    SA = ASPost(SA)
    
    # Fill capacities after shifting
    SA = FillCapacity(SA)
    
    return SA

In [48]:
SA = ShiftAction(SM)

## <font color = "brown"> Miscellaneous </font>

In [24]:
def CalculateRevenue(df):
    
    # Remove stationary points
    rev = df[(df["Status"] == "Charge") | (df["Status"] == "Disharge")].sort_values("Time")
    
    # Set marginal loss factors for each action
    df["mlf"] = np.where(df["Status"] == "Charge", 1/0.991, 0.991)
    
    # Ignoring signs
    df["Actual"] = np.where(df["Status"] == "Charge", -abs(df["Actual"]), abs(df["Actual"]))
    
    # Find revenue for each actions
    df["Revenue"] = df["Price"] * df["Actual"] * df["mlf"]
    
    # Sum up revenues
    revenue = df["Revenue"].sum()
    
    return revenue

In [25]:
def ValidityCheck(df):
    
    # Divide actions
    charge = df[df["Status"] == "Charge"]
    discharge = df[df["Status"] == "Discharge"]
    
    # Check if power used exceeds 300
    Condition0 = abs(df["Actual"].max()) > 150
    
    # Check if capacity exceeds 580
    Condition1 = df["Closing Capacity"].max() > 580
    
    # Check if capacity falls below 0
    Condition2 = df["Closing Capacity"].min() < 0
    
    # Check for duplicate time
    Condition3 = (len(df) != len(df.drop_duplicates("Time")))
    
    # Check for non-continuous time
    Condition4 = ((((df["Time"] - df["Time"].shift()).dt.total_seconds()/60) < 0).sum()) != 0
    
    # Check capacities validness
    Condition5 = (df["Opening Capacity"].dropna() != df["Closing Capacity"].dropna().shift()).sum() > 1
    
    # Decreasing charge
    Condition6 = len(df[(df["Status"] == "Charge") & ((df["Closing Capacity"] - df["Opening Capacity"]) < 0)]) != 0
    
    # Increasing discharge
    Condition7 = len(df[(df["Status"] == "Discharge") & ((df["Closing Capacity"] - df["Opening Capacity"]) > 0)]) != 0
    
    # Printed error messages
    ErrorMessages = ["Power Exceeds 300!",
                     "Capacity Exceeds 580!",
                     "Capacity Below 0!",
                     "Duplicate Records Found!",
                     "Non-Consecutive Time Data!",
                     "Capacities Chain Broken!",
                     "Charging Decreases Capacity!",
                     "Discharging Increases Capacity!"]
    
    # Boolean error list
    Errors = [Condition0, Condition1, Condition2, Condition3, 
              Condition4, Condition5, Condition6, Condition7]
    
    count = 0
    # Loop through all possible errors
    for e, m in zip(Errors, ErrorMessages):
        if e:
            # Print if error is true
            print(m)
            count += 1
            
    # Number of errors
    print(f"Number of Errors: {count}")

----

## <font color = "brown"> Main </font>

In [26]:
def main(n = 17, df = df):
    
    Model = MovingAverage(n, df = df)
    Model = Maximisation(Model, df = df)
    Model = LossRemoval(Model)
    Model = ShiftAction(Model)
    Model = Stationary(Model, df = df)
    Model = ShiftAction(Model)

    return Model

In [27]:
Prediction = main(n = 17, df = df)

In [28]:
ValidityCheck(Prediction.copy())
CalculateRevenue(Prediction.copy())

Number of Errors: 0


124913934.74379253

In [30]:
Prediction.tail(50)

,Time,Price,Status,Actual,Restrict,Opening Capacity,Closing Capacity,Period,ROCNACharge,ROCNADischarge,Shift
63407,2021-08-13 23:30:00,76.50,Do Nothing,-0.000000,0.0,150.0,150.0,24078,NaN,5087.0,True
63408,2021-08-14 00:00:00,76.38,Do Nothing,-0.000000,0.0,150.0,150.0,24078,NaN,5087.0,True
63409,2021-08-14 00:30:00,77.12,Do Nothing,-0.000000,0.0,150.0,150.0,24078,NaN,5087.0,True
63410,2021-08-14 01:00:00,87.96,Discharge,135.000000,-150.0,150.0,0.0,24079,NaN,5087.0,True
63411,2021-08-14 01:30:00,76.75,Do Nothing,-0.000000,0.0,0.0,0.0,24080,5088.0,5087.0,True
63412,2021-08-14 02:00:00,77.71,Do Nothing,-0.000000,0.0,0.0,0.0,24080,5088.0,5087.0,True
63413,2021-08-14 02:30:00,70.46,Do Nothing,-0.000000,0.0,0.0,0.0,24080,5088.0,5087.0,True
63414,2021-08-14 03:00:00,50.07,Do Nothing,-0.000000,0.0,0.0,0.0,24080,5088.0,5087.0,True
63415,2021-08-14 03:30:00,46.53,Do Nothing,-0.000000,0.0,0.0,0.0,24081,5088.0,NaN,True
63416,2021-08-14 04:00:00,37.60,Charge,-150.000000,135.0,0.0,135.0,24081,5088.0,NaN,True


----

# <font color = 'brown'> Bonus Task </font>

### <font color = 'brown'> Mandatory Task on Test Period </font>

In [88]:
test_start_period = '2021-07-01 00:00:00'
test_end_period   = '2021-08-15 23:30:00'

df_test = df[(df['Time'] >= test_start_period) & (df['Time'] <= test_end_period)]

In [89]:
df_test

,Time,Price
61296,2021-07-01 00:00:00,51.71
61297,2021-07-01 00:30:00,90.51
61298,2021-07-01 01:00:00,73.91
61299,2021-07-01 01:30:00,33.79
61300,2021-07-01 02:00:00,43.57
...,...,...
63452,2021-08-14 22:00:00,49.93
63453,2021-08-14 22:30:00,62.86
63454,2021-08-14 23:00:00,32.26
63455,2021-08-14 23:30:00,25.10


In [90]:
Test_Prediction = main(n = 17, df = df_test)

In [91]:
Test_Prediction.to_excel('../../preprocessed_data/Mandatory_Test_MainModel.xlsx')

### <font color = 'brown'>Vector Auto-Regression </font>

In [57]:
# Read file
df_VAR = pd.read_excel('../../prediction_data/VAR_predictions_log.xlsx')

In [58]:
df_VAR.columns = ['Time', 'Price']
df_VAR['Time'] = df_VAR['Time'].astype('datetime64[ns]')

In [59]:
time = df_VAR['Time']

In [60]:
# MA = MovingAverage(17, df = df_VAR)
# RM = Maximisation(MA, df = df_VAR)
# LR = LossRemoval(RM)
# SM = Stationary(LR, df = df_VAR)
# SA = ShiftAction(SM)

In [61]:
# Running mandatory task's algorithm using predicted price
VAR_Prediction = main(17, df = df_VAR)

In [62]:
VAR_Prediction

,Time,Price,Status,Actual,Restrict,Opening Capacity,Closing Capacity,Period,ROCNACharge,ROCNADischarge,Shift
0,2021-07-01 00:00:00,45.901292,Do Nothing,-0.0,0.0,0.0,0.0,1,1.0,NaN,True
1,2021-07-01 00:30:00,56.490789,Do Nothing,-0.0,0.0,0.0,0.0,1,1.0,NaN,True
2,2021-07-01 01:00:00,69.569596,Do Nothing,-0.0,0.0,0.0,0.0,1,1.0,NaN,True
3,2021-07-01 01:30:00,68.062152,Do Nothing,-0.0,0.0,0.0,0.0,1,1.0,NaN,True
4,2021-07-01 02:00:00,40.896315,Do Nothing,-0.0,0.0,0.0,0.0,1,1.0,NaN,True
...,...,...,...,...,...,...,...,...,...,...,...
2156,2021-08-14 22:00:00,58.635315,Do Nothing,-0.0,0.0,580.0,580.0,787,167.0,NaN,False
2157,2021-08-14 22:30:00,52.664372,Do Nothing,-0.0,0.0,580.0,580.0,787,167.0,NaN,False
2158,2021-08-14 23:00:00,50.100266,Do Nothing,-0.0,0.0,580.0,580.0,787,167.0,NaN,False
2159,2021-08-14 23:30:00,41.263355,Do Nothing,-0.0,0.0,580.0,580.0,787,167.0,NaN,False


In [63]:
VAR_Prediction['Price'] = actual_price

NameError: name 'actual_price' is not defined

In [ ]:
CalculateRevenue(VAR_Prediction)

In [ ]:
# Save
VAR_Prediction.to_excel('../../prediction_data/VAR_Prediction_MainModel.xlsx')

### <font color = 'brown'>ARMA</font>

In [ ]:
# Preprocessing
df_ARIMA = pd.read_excel('../../prediction_data/ARIMA_predictions.xlsx')
df_ARIMA.columns = ['Actual Price', 'Predicted Price']
df_ARIMA['Time'] = time

In [ ]:
df_ARIMA

In [ ]:
actual_price = df_ARIMA['Actual Price']

In [ ]:
df_ARIMA = df_ARIMA[['Predicted Price', 'Time']]

In [ ]:
df_ARIMA.columns = ['Price', 'Time']
df_ARIMA['Time'] = df_ARIMA['Time'].astype('datetime64[ns]')
df_ARIMA = df_ARIMA[['Time', 'Price']]

In [ ]:
ARIMA_Prediction = main(17, df = df_ARIMA)

In [ ]:
ARIMA_Prediction

In [ ]:
# Save
ARIMA_Prediction.to_excel('../../prediction_data/ARIMA_Prediction_MainModel.xlsx')

### <font color = 'brown'>Long Short-Term Memory </font>

In [ ]:
# Preprocessing
df_LSTM = pd.read_excel('../../prediction_data/LSTM_predictions.xlsx')
df_LSTM.columns = ['Actual Price', 'Predicted Price']
df_LSTM['Time'] = time

In [ ]:
actual_price = df_LSTM['Actual Price']

In [ ]:
df_LSTM = df_LSTM[['Predicted Price', 'Time']]

In [ ]:
df_LSTM.columns = ['Price', 'Time']
df_LSTM['Time'] = df_LSTM['Time'].astype('datetime64[ns]')
df_LSTM = df_LSTM[['Time', 'Price']]

In [ ]:
LSTM_Prediction = main(17, df = df_LSTM)

In [ ]:
# Converts back to actual price
LSTM_Prediction['Price'] = actual_price

In [ ]:
CalculateRevenue(LSTM_Prediction)

In [ ]:
# Save
LSTM_Prediction.to_excel('../../prediction_data/LSTM_Prediction_MainModel.xlsx')

----

## <font color = "brown"> Replacing MA </font>

In [35]:
gil = pd.read_excel("localMaximisation.xlsx") # Open original data
gil = gil[["Time (UTC+10)", "Regions VIC Trading Price ($/MWh)", "Market Dispatch (MWh)",
          "Opening Capacity (MWh)", "Closing Capacity (MWh)"]] # Select time and victoria prices
gil.columns = ["Time", "Price", "Actual", "Opening Capacity", "Closing Capacity"] # Rename columns
gil["Time"] = pd.to_datetime(df["Time"]) # Convert data type
gil = gil.sort_values("Time").reset_index(drop = True) # Finalise
gil.index += 1
gil = pd.merge(df["Time"], gil.drop("Time", axis = 1), left_index = True, right_index = True)

FileNotFoundError: [Errno 2] No such file or directory: 'localMaximisation.xlsx'

In [ ]:
gil["Restrict"] = - np.where(gil["Actual"] <= 0, gil["Actual"] * 0.9, gil["Actual"] / 0.9)
gil["Status"] = np.where(gil["Actual"] < 0, "Charge", np.where(gil["Actual"] > 0, "Discharge", "Do Nothing"))

In [153]:
df_LSTM

,Actual Price,Predicted Price,Time
0,90.51,39.546181,2021-07-01 00:00:00
1,73.91,42.148338,2021-07-01 00:30:00
2,33.79,44.220074,2021-07-01 01:00:00
3,43.57,42.094860,2021-07-01 01:30:00
4,45.90,39.311871,2021-07-01 02:00:00
...,...,...,...
2155,49.93,56.998489,NaN
2156,62.86,51.344746,NaN
2157,32.26,47.449478,NaN
2158,25.10,41.979435,NaN


In [ ]:
ValidityCheck(gil)

In [ ]:
gil[gil['Opening Capacity'] != gil['Closing Capacity'].shift()]

In [ ]:
FillCapacity(gil)

In [ ]:
def main2(model):
    
    model = Maximisation(model, Chronos = False)
    model = LossRemoval(model)
    model = ShiftAction(model)
    model = Stationary2(model)
    model = ShiftAction(model)
    
    return model

GLM = main2(gil.copy())

In [ ]:
ValidityCheck(GLM)
CalculateRevenue(GLM)

In [ ]:
# 128193560.2936161